<a href="https://colab.research.google.com/github/rohithpaul0304/Aerial-Cactus-Identification/blob/master/basic_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random
import json
import pickle


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
with open("intents.json") as file:
    data=json.load(file)

In [36]:
data

{'intents': [{'context_set': '',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Whats up'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'context_set': '',
   'patterns': ['cya',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Have a Good day'],
   'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
   'tag': 'goodbye'},
  {'context_set': '',
   'patterns': ['how old',
    'how old is tim',
    'what is your age',
    'how old are you',
    'age?'],
   'responses': ['I am 18 years old!', '18 years young!'],
   'tag': 'age'},
  {'context_set': '',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Tim.',
    "I'm Tim!",
    "I'm Tim aka Tech With Tim."],
   'tag': 'name'},
  {'context_set': '',
   'patterns': ['Id like to buy something',
    'whats on the men

In [0]:
words=[]
labels=[]
docs_x=[]
docs_y=[]

In [0]:
for intent in data["intents"]:
  for pattern in intent["patterns"]:
    wrds=nltk.word_tokenize(pattern)
    words.extend(wrds)
    docs_x.append(wrds)
    docs_y.append(intent["tag"])

  if intent["tag"] not in labels:
      labels.append(intent["tag"])

In [0]:
words=[stemmer.stem(w.lower()) for w in words if w != "?"]
words=sorted(list(set(words)))

In [0]:
labels=sorted(labels)

In [0]:
training=[]
output=[]

In [0]:
out_empty=[0 for _ in range(len(labels))]

In [0]:
for x,doc in enumerate(docs_x):
  bag = []
  wrds=[stemmer.stem(w) for w in doc]
  for w in words:
    if w in wrds:
      bag.append(1)
    else:
      bag.append(0)

  output_row = out_empty[:]
  output_row[labels.index(docs_y[x])] = 1

  training.append(bag)
  output.append(output_row)


In [0]:
training = numpy.array(training)
output = numpy.array(output)

with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

In [0]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


In [46]:
model.fit(training, output, n_epoch=500, batch_size=16, show_metric=True)
model.save("model.tflearn")


Training Step: 999  | total loss: 0.08034 | time: 0.004s
| Adam | epoch: 500 | loss: 0.08034 - acc: 1.0000 -- iter: 16/26
Training Step: 1000  | total loss: 0.08187 | time: 0.008s
| Adam | epoch: 500 | loss: 0.08187 - acc: 1.0000 -- iter: 26/26
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [0]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

In [48]:

def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: Hello
Good to see you again!
You: I'd like to buy something
Cookies are on the menu!
You: when are you guys open
We are open 7am-4pm Monday-Friday!
You: goodbve
Hello!
You: cya
Sad to see you go :(
You: hey
Good to see you again!
You: whars up
Hello!
You: how are you
Good to see you again!
You: whats on the menu
Cookies are on the menu!
You: how much do they cost
Good to see you again!
You: quit
